# Distance to edge tool 

This code creates a mathematical edge of the tumor using dapi position data, then calculates the shortest distance to the border from each dapi position adding the results as a new column. Data used must contain X and Y dapi postion.

In [1]:
import os
import glob
import numpy as np
import pandas as pd
from scipy.spatial import ConvexHull

# 1. Find all CSV files in the folder
csv_files = glob.glob('csv_for_tools/*.csv')

# 2. Error if none found
if not csv_files:
    raise FileNotFoundError("No CSV files found in 'csv_for_tools/'")

# 3. Pick the first CSV
first_csv = csv_files[0]
print(f"Reading: {first_csv}")

# 4. Read into a DataFrame
df = pd.read_csv(first_csv)

# (Optional) Inspect the top of the DataFrame
df.head()


Reading: csv_for_tools/merged_for_tools.csv


/var/folders/bv/7wzy33ls6ln4ydwr6s8vwq8h0000gn/T/ipykernel_45676/992411655.py:19: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(first_csv)


,POSITION X,POSITION Y,POSITION Z,UNIT,CATEGORY,COLLECTION,TIME,SET 1,USELESS,ID,...,CD206 OFF F480,CD4,LY6G,F480,CD3,NK1.1,CD11B,CD8,MHCII,Distance_to_Edge
0,71200.969,45685.188,-22,µm,Surface,Position,1,NaN,NaN,38172,...,0,0,0,0,0,0,0,0,0,181.460554
1,72096.891,44789.438,-19,µm,Surface,Position,1,NaN,NaN,38173,...,0,0,0,0,0,0,0,0,0,670.734247
2,71387.711,45500.258,-20,µm,Surface,Position,1,NaN,NaN,38174,...,0,0,0,0,0,0,0,0,0,366.535908
3,71587.172,45303.039,-24,µm,Surface,Position,1,NaN,NaN,38175,...,0,0,0,0,0,0,0,0,0,563.910161
4,71925.766,44961.535,-19,µm,Surface,Position,1,NaN,NaN,38176,...,0,0,0,0,0,0,0,0,0,698.622670


# Check data before contuining

Make sure there is a positon x and position y column otherwise code will not work that is all. 

In [2]:

pts = df[['POSITION X', 'POSITION Y']].values             # shape (N,2) array

# 2) Compute the convex hull
hull = ConvexHull(pts)

# hull.simplices is an (E,2) array of index pairs (i, j)
# each representing an edge between pts[i] and pts[j].
edges = hull.simplices

# 3) Define a vectorized distance‐to‐segment function
def point_to_segment_distances(points, seg_start, seg_end):
    """
    points: (N,2) array
    seg_start, seg_end: each (2,) array
    returns: (N,) array of distances from each point to the segment
    """
    v = seg_end - seg_start                      # (2,)
    w = points - seg_start                       # (N,2)
    # projection factor of each point onto the infinite line
    t = np.einsum('ij,j->i', w, v) / np.dot(v, v)
    # clamp to [0,1] so we “snap” to the segment
    t_clamped = np.clip(t, 0.0, 1.0)             # (N,)
    proj = seg_start + t_clamped[:,None] * v     # (N,2)
    # euclidean distance from each point to its projection
    return np.linalg.norm(points - proj, axis=1)

# 4) Loop over edges, keep the minimum distance per point
all_dists = np.full(len(pts), np.inf)
for i, j in edges:
    start, end = pts[i], pts[j]
    d = point_to_segment_distances(pts, start, end)
    all_dists = np.minimum(all_dists, d)

# 5) Attach back to your DataFrame
df['Distance_to_Edge'] = all_dists

# 6) (Optional) Inspect
df

,POSITION X,POSITION Y,POSITION Z,UNIT,CATEGORY,COLLECTION,TIME,SET 1,USELESS,ID,...,CD206 OFF F480,CD4,LY6G,F480,CD3,NK1.1,CD11B,CD8,MHCII,Distance_to_Edge
0,71200.969,45685.188,-22,µm,Surface,Position,1,NaN,NaN,38172,...,0,0,0,0,0,0,0,0,0,181.460554
1,72096.891,44789.438,-19,µm,Surface,Position,1,NaN,NaN,38173,...,0,0,0,0,0,0,0,0,0,670.734247
2,71387.711,45500.258,-20,µm,Surface,Position,1,NaN,NaN,38174,...,0,0,0,0,0,0,0,0,0,366.535908
3,71587.172,45303.039,-24,µm,Surface,Position,1,NaN,NaN,38175,...,0,0,0,0,0,0,0,0,0,563.910161
4,71925.766,44961.535,-19,µm,Surface,Position,1,NaN,NaN,38176,...,0,0,0,0,0,0,0,0,0,698.622670
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
722318,69372.898,44787.383,-19,µm,Surface,Position,1,NaN,Class A,116161,...,0,0,0,0,0,0,0,0,0,764.198882
722319,69422.750,44695.879,-19,µm,Surface,Position,1,NaN,Class A,126618,...,0,0,0,0,0,0,1,0,0,863.856988
722320,69426.352,44698.328,-23,µm,Surface,Position,1,NaN,Class A,126637,...,0,0,0,0,0,0,1,0,0,862.182148
722321,69428.562,44454.734,-20,µm,Surface,Position,1,NaN,Class A,128048,...,0,0,0,0,0,0,1,0,0,1101.246850


# Review the ouput above before continuing

If the output above looks correct then move to next cell below. Running the next cell will update the csv file and rewrite it by adding the edge distance column make sure you are ok with this before running next cell. 

In [29]:
csv_path = glob.glob('csv_for_tools/*.csv')[0]

df.to_csv(csv_path, index=False)